In [4]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bab08850-6eb2-414e-801e-1c78488e3a90;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 639ms :: artifacts dl 28ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [5]:
%env S3ADRESS=url
%env PASSWORD_SQL=PASSWORD
%env USER_SQL=USER

env: S3ADRESS=s3a://daniel-3cco-poupay-poupay-bucket-estruturados
env: PASSWORD_SQL=PASSWORD
env: USER_SQL=USER


In [6]:
import os

S3_address = os.environ['S3ADRESS']
password_sql = os.environ['PASSWORD_SQL']
user_sql = os.environ['USER_SQL']

In [8]:
df_unidade_federativa = spark.read.option("multiline","true")\
                        .json(f"{S3_address}/dados-brutos/estados_soja_1974_2020.json")\
                        .toPandas()
df_regiao = spark.read.option('delimiter', ',')\
    .option('header', 'true').csv(f"{S3_address}/dados-brutos/estados.csv")\
    .toPandas()

22/05/31 00:52:41 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [10]:
df_unidade_federativa.dropna(inplace=True)
df = df_unidade_federativa.merge(df_regiao, left_on='Unidade da Federação'
                            , right_on='nome')[['Ano', 'Toneladas', 'Unidade da Federação','regiao']]

,Ano,Toneladas,Unidade da Federação,regiao
0,1974.0,-,Rondônia,Norte
1,1975.0,-,Rondônia,Norte
2,1976.0,-,Rondônia,Norte
3,1977.0,-,Rondônia,Norte
4,1978.0,-,Rondônia,Norte
...,...,...,...,...
1264,2016.0,233888,Distrito Federal,Centro-Oeste
1265,2017.0,241500,Distrito Federal,Centro-Oeste
1266,2018.0,253110,Distrito Federal,Centro-Oeste
1267,2019.0,261000,Distrito Federal,Centro-Oeste


In [4]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

_df = spark.createDataFrame(df)

_df = _df.select(
    col('Ano').cast(IntegerType()).alias('ANO'),
    col('Toneladas').cast(IntegerType()).alias('TONELADAS'),
    col('Unidade da Federação').alias('UF'),
    col('regiao').alias('REGIAO')
)

_df.write \
   .mode('overwrite') \
   .csv('s3a://daniel-3cco-poupay-poupay-bucket-estruturados/dados-tratados/brazil_soybean_regiao')

22/05/30 02:12:44 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 02:12:45 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 02:12:45 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [5]:
dataframe = spark.read.csv(f"{S3_address}/dados-tratados/brazil_soybean_regiao")

In [6]:
dataframe = dataframe.withColumnRenamed("_c0", "Ano")
dataframe = dataframe.withColumnRenamed("_c1", "Toneladas")
dataframe = dataframe.withColumnRenamed("_c2", "UF")
dataframe = dataframe.withColumnRenamed("_c3", "Regiao")
dataframe_pandas = dataframe.toPandas()

In [7]:
dataframe_pandas = dataframe_pandas.fillna(0).astype({"Toneladas": int})

In [8]:
dataframe_pandas[(dataframe_pandas['Regiao'] == 'Centro-Oeste') & (dataframe_pandas['Ano'] == '1974')]

,Ano,Toneladas,UF,Regiao
447,1974,0,Mato Grosso do Sul,Centro-Oeste
494,1974,307010,Mato Grosso,Centro-Oeste
541,1974,99000,Goiás,Centro-Oeste
588,1974,23,Distrito Federal,Centro-Oeste


In [9]:
sum_dataframe_groupby = dataframe_pandas.groupby(['Regiao','Ano']).sum().reset_index()

In [10]:
sum_dataframe_groupby

,Regiao,Ano,Toneladas
0,Centro-Oeste,1974,406033
1,Centro-Oeste,1975,346016
2,Centro-Oeste,1976,339177
3,Centro-Oeste,1977,785011
4,Centro-Oeste,1978,579941
...,...,...,...
230,Sul,2016,35471797
231,Sul,2017,40373389
232,Sul,2018,38912491
233,Sul,2019,37116995


In [13]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

_df = spark.createDataFrame(sum_dataframe_groupby)

_df = _df.select(
    col('Ano').cast(IntegerType()).alias('ANO'),
    col('Toneladas').cast(IntegerType()).alias('TONELADAS'),
    col('Regiao').alias('REGIAO')
)

_df.write \
   .mode('overwrite') \
   .csv(f'{S3_address}/dados-cliente/brazil_soybean_regiao')

_df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://poupay-01.database.windows.net;databaseName=poupay;') \
 .option('dbtable', 'REGIAO_SOJA') \
 .option('user', user_sql) \
 .option('password', password_sql) \
 .save()

22/05/30 02:19:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 02:19:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/30 02:19:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
